### Object initiation
The object below will be initiated with all the attributes and functions needed to store graphs  
Based on: https://datavalet.atlassian.net/wiki/spaces/PM/pages/1626308609/Counting+with+position

In [58]:
import networkx as nx
import math
import pandas as pd
import numpy as np
import itertools
from itertools import chain

### Loading functions
- Loading the window function
- Loading the distance computing logic

In [12]:
import sys  
sys.path.insert(0, './Functions/')

import Window
import Distance

In [13]:
def distance_function(fspl, frequency):
    '''
    Returns a distance in meter from the parameters:
    fspl: basically the signal strength observed in absolute value
    frequency: the frequency in ghz
    '''
    return 10**((fspl-(20*math.log(frequency, 10))-32.45)/20)

In [71]:
class Position_graph:
    def __new__(cls, *args, **kwargs):
        print(f"New empty graph")
        return super().__new__(cls)

    def __init__(self, dataset, sorting_columns = ['Timestamp','device_mac','ap_mac']):
        self.dataset = dataset.sort_values(sorting_columns).reset_index(drop=True)
        print(f"Length of dataset: {dataset.shape[0]}")
        self.walking_param = 1.42
        self.distance_formula = distance_function
        self.channel_table = pd.read_csv(f'channel.csv')
        #Initiating complex structures
        
        self.window_kwargs = {
            'min_window_len':3,
            'min_value_len':1,
        }
        
        self.distances = ''
        self.active_distance_slice = ''
        self.graph = 'Please initiate the graph using the function generate_graph_structure()'
        self.subgraph = 'Please initiate the graph using the function generate_subgraphs()'
        self.topological_order = 'Please initiate the graph using the function generate_topological_order()'
        
        
    def compute_window(self, **kwargs):
        self.window_index = Window.generate_window_dict(self.dataset, 'Timestamp', window_size=1800, window_frequency=900, **kwargs)
        
    def compute_distance_in_windows(self, slice_amount=100):
        dummydataset = self.dataset
        dummydataset['signal_strength'] = self.dataset[['ap_mac','signal_strength', 'channel']].values.tolist()

        kwargs = {
         'min_window_len':2,
         'min_value_len':1   
        }
        window_index = Window.generate_window_dict(self.dataset, 'Timestamp', window_size=1800, window_frequency=900, **self.window_kwargs)
        sub_window_index = dict(itertools.islice(window_index.items(), slice_amount))
        df_list = {}

        for key, value in sub_window_index.items():
            df_list[key] = self.dataset[value[0]:value[1]].groupby(['device_mac'])['signal_strength'].apply(lambda x: list(np.unique(x))).apply(list).reset_index()
        
        #computing the distance by observation and by ap
        final_dict = dict((k, Distance.transform_dict_entry(v, 'signal_strength', 'distance_by_ap', Distance.cycle_and_apply)) for k,v in df_list.items())  
        #exploding the array produced into a distance column and coresponding ap_name column and the numbers of ap encountered
        final_dict = dict((k, Distance.explode_outer(v, 'distance_by_ap', ['distance','ap_name','number_ap'])) for k,v in final_dict.items())
        #cleaning the resulting array
        final_dict = dict((k, v.drop(['signal_strength', 'distance_by_ap'], axis=1)) for k,v in final_dict.items())
        #replacing the device_mac by a true random uuid in order to avoid breaking the next algorithms
        final_dict = dict((k, Distance.transform_dict_entry(v, 'device_mac', 'device_mac', Distance.generate_uuid)) for k,v in final_dict.items()) 
        self.active_distance_slice = final_dict
        print(f'Slice of {slice_amount} computed for distance stored under the active_distance_slice attribute')
    
    def compute_distance_1_to_n(self, n_window=5, n_starting_index = 0):
        if self.active_distance_slice == '':
            raise Exception('Please compute compute_distance_in_windows before this function!')
        #storing the keys only
        dict_keys = list(self.active_distance_slice.keys())
        first_item = Distance.fetch_one_item(self.active_distance_slice, dict_keys[0], n_starting_index)
        kwargs = {
            'discriminant_walking' : self.walking_param,
            'time_window_length' : dict_keys[0][1]-dict_keys[0][0],
            'master_dict' : self.active_distance_slice,
            'item' : first_item
        }
        dicts =  Distance.compute_n_windows(dict_keys[1:n_window], **kwargs)
        tuple_dataframe = pd.DataFrame.from_dict(list(chain.from_iterable(dicts)))
        tuple_dataframe
        return tuple_dataframe
    
    def generate_graph_structure(self):
        return None
    
    def generate_topological_order(self):
        return None
    
    def generate_subgraphs(self):
        return None
        
    def __repr__(self) -> str:
        return f"{type(self).__name__} is a graph structure with the following attributes:"

In [60]:
%%bigquery df_onroute_position_window
Select *
from data-prod-270222.datascience.position_raw

Downloading: 100%|██████████| 82228/82228 [00:00<00:00, 144380.69rows/s]


In [72]:
mygraph = Position_graph(df_onroute_position_window)

New empty graph
Length of dataset: 82228


In [73]:
mygraph.compute_distance_in_windows()

Slice of 100 computed for distance stored under the active_distance_slice attribute


In [74]:
mygraph.compute_distance_1_to_n()

./Functions/Distance.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df['ap_name_bool'] = working_df['ap_name'].apply(lambda x: compare_set_column(x,item['ap_name']))


,source_device_mac,device_mac,distance,ap_name,number_ap
0,57176f53-51ca-4fc2-b9ee-39203e044bb8,3414a956-5c46-490a-a3a0-93dd4e85b442,[0.0],[CCD083CF7016],1
1,57176f53-51ca-4fc2-b9ee-39203e044bb8,c52a0ea8-7e18-4ea8-84fc-10fdb4dd7add,[0.12457528788493244],[CCD083CF7016],1
2,57176f53-51ca-4fc2-b9ee-39203e044bb8,d31bed3d-57e0-4a29-ab79-75d6c27eb44f,[2.2328935026736376],[CCD083CF7016],1
3,57176f53-51ca-4fc2-b9ee-39203e044bb8,019de57e-8fd3-458a-8aa8-0d6b1c7cf2fd,[0.31187315594842224],[CCD083CF7016],1
4,57176f53-51ca-4fc2-b9ee-39203e044bb8,3eb964dc-f832-4a4d-81a3-a676a59aff13,[4.700509134820309],[CCD083CF7016],1
5,57176f53-51ca-4fc2-b9ee-39203e044bb8,031f6f71-9a00-4466-af73-b39b194846dd,[0.040008659387058776],[CCD083CF7016],1
6,57176f53-51ca-4fc2-b9ee-39203e044bb8,205d67ba-b2bb-4cb4-8c77-52cd600d7abf,[0.12457528788493244],[CCD083CF7016],1
7,57176f53-51ca-4fc2-b9ee-39203e044bb8,c7a07d7c-9244-4a9a-9480-fba669572c17,[5.858947077022713],[CCD083CF7016],1
8,57176f53-51ca-4fc2-b9ee-39203e044bb8,972e5186-59af-4fd7-8df7-e0506d924333,[4.035951221182009],[CCD083CF7016],1
9,57176f53-51ca-4fc2-b9ee-39203e044bb8,2ccf853f-6b93-4c48-b252-3362cc550097,[7.741171186388208],[CCD083CF7016],1


In [11]:
mygraph.window_index

{(1637329500, 1637329503): (0, 99),
 (1637329504, 1637329507): (100, 170),
 (1637329508, 1637329511): (171, 243),
 (1637329512, 1637329514): (244, 344),
 (1637329515, 1637329518): (345, 462),
 (1637329519, 1637329521): (463, 536),
 (1637329522, 1637329525): (537, 610),
 (1637329526, 1637329528): (611, 637),
 (1637329529, 1637329532): (638, 713),
 (1637329533, 1637329535): (714, 752),
 (1637329536, 1637329539): (753, 824),
 (1637329540, 1637329542): (825, 855),
 (1637329543, 1637329546): (856, 951),
 (1637329547, 1637329549): (952, 996),
 (1637329550, 1637329553): (997, 1076),
 (1637329554, 1637329556): (1077, 1116),
 (1637329557, 1637329560): (1117, 1210),
 (1637329561, 1637329563): (1211, 1268),
 (1637329564, 1637329567): (1269, 1330),
 (1637329568, 1637329570): (1331, 1423),
 (1637329571, 1637329574): (1424, 1529),
 (1637329575, 1637329577): (1530, 1641),
 (1637329578, 1637329581): (1642, 1692),
 (1637329582, 1637329584): (1693, 1783),
 (1637329585, 1637329588): (1784, 1962),
 (16373

In [22]:
mygraph.dataset

,device_mac,ap_oem,channel,Timestamp_ms,Timestamp,local_time,is_associated,signal_strength,noise_floor,channel_1,brand_name,market_segment,ap_mac,site_uuid
0,462E2AC8A109,Aruba Generic,11,1637333100696,1637333100,2021-11-19 14:45:00+00:00,False,-70,96,11,Farm Boy,Food-Supermarket,CCD083CF7016,657b300b-db2e-f810-ffc6-e3a84b96192c
1,56D97A9C14EB,Aruba Generic,140,1637333100095,1637333100,2021-11-19 14:45:00+00:00,True,-57,94,140,Farm Boy,Food-Supermarket,CCD083CF729E,657b300b-db2e-f810-ffc6-e3a84b96192c
2,AAFEA44A6DF7,Aruba Generic,11,1637333100039,1637333100,2021-11-19 14:45:00+00:00,False,-48,96,11,Farm Boy,Food-Supermarket,CCD083CF7016,657b300b-db2e-f810-ffc6-e3a84b96192c
3,AAFEA44A6DF7,Aruba Generic,11,1637333100448,1637333100,2021-11-19 14:45:00+00:00,False,-48,96,11,Farm Boy,Food-Supermarket,CCD083CF7016,657b300b-db2e-f810-ffc6-e3a84b96192c
4,AAFEA44A6DF7,Aruba Generic,11,1637333100836,1637333100,2021-11-19 14:45:00+00:00,False,-48,96,11,Farm Boy,Food-Supermarket,CCD083CF7016,657b300b-db2e-f810-ffc6-e3a84b96192c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82422,9A173523AAB9,Aruba Generic,140,1637336700000,1637336700,2021-11-19 15:45:00+00:00,False,-80,94,140,Farm Boy,Food-Supermarket,CCD083CF729E,657b300b-db2e-f810-ffc6-e3a84b96192c
82423,AE6DD67E8D62,Aruba Generic,11,1637336700927,1637336700,2021-11-19 15:45:00+00:00,False,-72,96,11,Farm Boy,Food-Supermarket,CCD083CF729E,657b300b-db2e-f810-ffc6-e3a84b96192c
82424,CCD083770170,Aruba Generic,52,1637336700567,1637336700,2021-11-19 15:45:00+00:00,True,-70,92,52,Farm Boy,Food-Supermarket,CCD083CF729E,657b300b-db2e-f810-ffc6-e3a84b96192c
82425,CCD083770170,Aruba Generic,52,1637336700220,1637336700,2021-11-19 15:45:00+00:00,True,-70,92,52,Farm Boy,Food-Supermarket,CCD083CF729E,657b300b-db2e-f810-ffc6-e3a84b96192c
